
# Pairwise alignment
(partly based on the materials of IAB: https://github.com/caporaso-lab/An-Introduction-To-Applied-Bioinformatics) , https://github.com/applied-bioinformatics/iab2/blob/main/book/pairwise-alignment.md 



In [ ]:
!pip install scikit-bio

In [ ]:
#Imports
from __future__ import division, print_function
import numpy as np
#disclaimer

from IPython.core.display import HTML
from IPython.core import page


from skbio.sequence import Protein
from skbio.alignment import TabularMSA
from skbio.alignment import global_pairwise_align, local_pairwise_align
from skbio.alignment._pairwise import _compute_score_and_traceback_matrices

from IPython.core.interactiveshell import InteractiveShell



#page.page = print
InteractiveShell.ast_node_interactivity = "all"
%pylab inline

   

In [ ]:
import tabulate

def show_F(h_sequence, v_sequence, data, hide_zeros=False, nonzero_val=None):
    rows = []
    col_headers = [c.decode('UTF-8') for c in h_sequence.values]
    row_headers = [c.decode('UTF-8') for c in v_sequence.values]
    pad_headers = data.shape == (len(row_headers) + 1, len(col_headers) + 1)
    if pad_headers:
        row_headers = [" "] + row_headers
        col_headers = [" "] + col_headers
    for h, d in zip(row_headers, data):
        current_row = [h]
        for e in d:
            if e == 0:
                if hide_zeros:
                    current_row.append('')
                else:
                    current_row.append(0)
            else:
                if nonzero_val is not None:
                    current_row.append(nonzero_val)
                else:
                    current_row.append(e)
        rows.append(current_row)
    return tabulate.tabulate(rows, headers=col_headers, tablefmt='html')

def show_substitution_matrix(headers, data):
    rows = []
    for h, d in zip(headers, data):
        current_row = [h]
        for e in d:
            current_row.append(e)
        rows.append(current_row)
    return tabulate.tabulate(rows, headers=headers, tablefmt='html')

def show_T(h_sequence, v_sequence, data):
    if data.dtype == int:
        data_ = T = np.full(shape=data.shape, fill_value=" ", dtype=str)
        translation_table = {0: "•", 1: "↖", 2: "↑", 3: "←"}
        for i, row in enumerate(data):
            for j, value in enumerate(row):
                data_[i, j] =  translation_table[value]
    else:
        data_ = data
    return show_F(h_sequence, v_sequence, data_)

def show_tabular_alignment_via_file(alignment_msa):
    '''
    Prints a long alignment in clustal format
    '''
    import string
    import random
    N=6
    random_fname = '/tmp/' + ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
    alignment_msa.write(random_fname,format='clustal')
    for line in open(random_fname):
        print(line.strip())

blosum50 = {'A': {'A': 5, 'C': -1, 'D': -2, 'E': -1, 'F': -3, 'G': 0, 'H': -2, 'I': -1, 'K': -1, 'L': -2, 'M': -1, 'N': -1, 'P': -1, 'Q': -1, 'R': -2, 'S': 1, 'T': 0, 'V': 0, 'W': -3, 'Y': -2},
'C': {'A': -1, 'C': 13, 'D': -4, 'E': -3, 'F': -2, 'G': -3, 'H': -3, 'I': -2, 'K': -3, 'L': -2, 'M': -2, 'N': -2, 'P': -4, 'Q': -3, 'R': -4, 'S': -1, 'T': -1, 'V': -1, 'W': -5, 'Y': -3},
'D': {'A': -2, 'C': -4, 'D': 8, 'E': 2, 'F': -5, 'G': -1, 'H': -1, 'I': -4, 'K': -1, 'L': -4, 'M': -4, 'N': 2, 'P': -1, 'Q': 0, 'R': -2, 'S': 0, 'T': -1, 'V': -4, 'W': -5, 'Y': -3},
'E': {'A': -1, 'C': -3, 'D': 2, 'E': 6, 'F': -3, 'G': -3, 'H': 0, 'I': -4, 'K': 1, 'L': -3, 'M': -2, 'N': 0, 'P': -1, 'Q': 2, 'R': 0, 'S': -1, 'T': -1, 'V': -3, 'W': -3, 'Y': -2},
'F': {'A': -3, 'C': -2, 'D': -5, 'E': -3, 'F': 8, 'G': -4, 'H': -1, 'I': 0, 'K': -4, 'L': 1, 'M': 0, 'N': -4, 'P': -4, 'Q': -4, 'R': -3, 'S': -3, 'T': -2, 'V': -1, 'W': 1, 'Y': 4},
'G': {'A': 0, 'C': -3, 'D': -1, 'E': -3, 'F': -4, 'G': 8, 'H': -2, 'I': -4, 'K': -2, 'L': -4, 'M': -3, 'N': 0, 'P': -2, 'Q': -2, 'R': -3, 'S': 0, 'T': -2, 'V': -4, 'W': -3, 'Y': -3},
'H': {'A': -2, 'C': -3, 'D': -1, 'E': 0, 'F': -1, 'G': -2, 'H': 10, 'I': -4, 'K': 0, 'L': -3, 'M': -1, 'N': 1, 'P': -2, 'Q': 1, 'R': 0, 'S': -1, 'T': -2, 'V': -4, 'W': -3, 'Y': 2},
'I': {'A': -1, 'C': -2, 'D': -4, 'E': -4, 'F': 0, 'G': -4, 'H': -4, 'I': 5, 'K': -3, 'L': 2, 'M': 2, 'N': -3, 'P': -3, 'Q': -3, 'R': -4, 'S': -3, 'T': -1, 'V': 4, 'W': -3, 'Y': -1},
'K': {'A': -1, 'C': -3, 'D': -1, 'E': 1, 'F': -4, 'G': -2, 'H': 0, 'I': -3, 'K': 6, 'L': -3, 'M': -2, 'N': 0, 'P': -1, 'Q': 2, 'R': 3, 'S': 0, 'T': -1, 'V': -3, 'W': -3, 'Y': -2},
'L': {'A': -2, 'C': -2, 'D': -4, 'E': -3, 'F': 1, 'G': -4, 'H': -3, 'I': 2, 'K': -3, 'L': 5, 'M': 3, 'N': -4, 'P': -4, 'Q': -2, 'R': -3, 'S': -3, 'T': -1, 'V': 1, 'W': -2, 'Y': -1},
'M': {'A': -1, 'C': -2, 'D': -4, 'E': -2, 'F': 0, 'G': -3, 'H': -1, 'I': 2, 'K': -2, 'L': 3, 'M': 7, 'N': -2, 'P': -3, 'Q': 0, 'R': -2, 'S': -2, 'T': -1, 'V': 1, 'W': -1, 'Y': 0},
'N': {'A': -1, 'C': -2, 'D': 2, 'E': 0, 'F': -4, 'G': 0, 'H': 1, 'I': -3, 'K': 0, 'L': -4, 'M': -2, 'N': 7, 'P': -2, 'Q': 0, 'R': -1, 'S': 1, 'T': 0, 'V': -3, 'W': -4, 'Y': -2},
'P': {'A': -1, 'C': -4, 'D': -1, 'E': -1, 'F': -4, 'G': -2, 'H': -2, 'I': -3, 'K': -1, 'L': -4, 'M': -3, 'N': -2, 'P': 10, 'Q': -1, 'R': -3, 'S': -1, 'T': -1, 'V': -3, 'W': -4, 'Y': -3},
'Q': {'A': -1, 'C': -3, 'D': 0, 'E': 2, 'F': -4, 'G': -2, 'H': 1, 'I': -3, 'K': 2, 'L': -2, 'M': 0, 'N': 0, 'P': -1, 'Q': 7, 'R': 1, 'S': 0, 'T': -1, 'V': -3, 'W': -1, 'Y': -1},
'R': {'A': -2, 'C': -4, 'D': -2, 'E': 0, 'F': -3, 'G': -3, 'H': 0, 'I': -4, 'K': 3, 'L': -3, 'M': -2, 'N': -1, 'P': -3, 'Q': 1, 'R': 7, 'S': -1, 'T': -1, 'V': -3, 'W': -3, 'Y': -1},
'S': {'A': 1, 'C': -1, 'D': 0, 'E': -1, 'F': -3, 'G': 0, 'H': -1, 'I': -3, 'K': 0, 'L': -3, 'M': -2, 'N': 1, 'P': -1, 'Q': 0, 'R': -1, 'S': 5, 'T': 2, 'V': -2, 'W': -4, 'Y': -2},
'T': {'A': 0, 'C': -1, 'D': -1, 'E': -1, 'F': -2, 'G': -2, 'H': -2, 'I': -1, 'K': -1, 'L': -1, 'M': -1, 'N': 0, 'P': -1, 'Q': -1, 'R': -1, 'S': 2, 'T': 5, 'V': 0, 'W': -3, 'Y': -2},
'V': {'A': 0, 'C': -1, 'D': -4, 'E': -3, 'F': -1, 'G': -4, 'H': -4, 'I': 4, 'K': -3, 'L': 1, 'M': 1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -2, 'T': 0, 'V': 5, 'W': -3, 'Y': -1},
'W': {'A': -3, 'C': -5, 'D': -5, 'E': -3, 'F': 1, 'G': -3, 'H': -3, 'I': -3, 'K': -3, 'L': -2, 'M': -1, 'N': -4, 'P': -4, 'Q': -1, 'R': -3, 'S': -4, 'T': -3, 'V': -3, 'W': 15, 'Y': 2},
'Y': {'A': -2, 'C': -3, 'D': -3, 'E': -2, 'F': 4, 'G': -3, 'H': 2, 'I': -1, 'K': -2, 'L': -1, 'M': 0, 'N': -2, 'P': -3, 'Q': -1, 'R': -1, 'S': -2, 'T': -2, 'V': -1, 'W': 2, 'Y': 8}}

aas = list(blosum50.keys())
aas.sort()
data = []
for aa1 in aas:
    row = []
    for aa2 in aas:
        row.append(blosum50[aa1][aa2])
    data.append(row)

aa_labels = ''.join(aas)

# Global alignment (NW)


Next, we're going to work with the standard algorithm for aligning a pair of biological sequences. This algorithm was originally published by [Saul B. Needleman and Christian D. Wunsch in 1970](https://www.ncbi.nlm.nih.gov/pubmed/5420325), and is therefore referred to as *Needleman-Wunsch alignment*. This performs what is known as *global alignment*, meaning that both sequences are aligned from their first residue (or base) through their last residue (or base). Later in this chapter, we'll contrast this with local alignment.

In [ ]:
seq1 = Protein("HEAGAWGHEE")
seq2 = Protein("PAWHEAE")

seq1 = TabularMSA([seq1])
seq2 = TabularMSA([seq2])

nw_matrix, traceback_matrix = _compute_score_and_traceback_matrices(
    seq1, seq2, 8, 8, blosum50)

HTML(show_F(seq1[0], seq2[0], nw_matrix))
HTML(show_T(seq1[0], seq2[0], traceback_matrix))


In [ ]:
# It is not enough! Traceback
from skbio.alignment._pairwise import _traceback
#%psource _traceback
aln1, aln2, score, _, _ = _traceback(traceback_matrix,nw_matrix,seq1,seq2, nw_matrix.shape[0]-1, nw_matrix.shape[1]-1)
print(aln1[0])
print(aln2[0])
print(score)


Let's check the documentation for the alignment program:

https://scikit.bio/docs/dev/generated/skbio.alignment.global_pairwise_align.html

In [ ]:
aln, score, _ = global_pairwise_align(Protein("HEAGAWGHEE"), Protein("PAWHEAE"), 
                                     8, 8, blosum50, penalize_terminal_gaps=True)

print(aln)
print(score)

# Local alignment - Smith-Waterman

In [ ]:
from skbio.alignment._pairwise import _init_matrices_sw

seq1 = TabularMSA([Protein("HEAGAWGHEE")])
seq2 = TabularMSA([Protein("PAWHEAE")])

sw_matrix, traceback_matrix = _compute_score_and_traceback_matrices(
    seq1, seq2, 5, 5, blosum50, new_alignment_score=0.0,
    init_matrices_f=_init_matrices_sw)

HTML(show_F(seq1[0], seq2[0], sw_matrix))
HTML(show_T(seq1[0], seq2[0], traceback_matrix))

In [ ]:
#traceback 
#finding maximum value (best local match)
max_value = 0.0
max_i = 0
max_j = 0
for i in range(sw_matrix.shape[0]):
    for j in range(sw_matrix.shape[1]):
        if sw_matrix[i, j] > max_value:
            max_i, max_j = i, j
            max_value = sw_matrix[i, j]

aln1, aln2, score, start_a1, start_a2 = _traceback(traceback_matrix, sw_matrix, seq1, seq2, max_i, max_j)
print(aln1[0])
print(aln2[0])
print(score)

Putting the pieces tohether
Let's check the documentation for the local alignment program:
https://scikit.bio/docs/dev/generated/skbio.alignment.local_pairwise_align.html


In [ ]:
# local alignment
aln, score, _ = local_pairwise_align(Protein("HEAGAWGHEE"), Protein("PAWHEAE"), 
                                     5, 5, blosum50)
print(aln)
print(score)

# USING AFFINE GAP penalty (in real world applications)


In the examples mentioned above all gaps are scored equally whether they represent the opening of a new insertion/deletion, or the extension of an existing insertion/deletion. This isn't ideal based on what we know about how insertion/deletion events occur (see [this discussion of replication slippage](http://www.ncbi.nlm.nih.gov/books/NBK21114/) if you're not familiar with the biological process that is thought to lead to small insertions as deletions). Instead, we might want to incur a large penalty for opening a gap, but a smaller penalty for extending an existing gap. This is referred to as *affine gap scoring*.

To score gap extensions differently from gap creations (or gap opens), we need to modify the terms corresponding to the addition of gaps in our scoring function. When we compute the score corresponding to a gap in our alignment (i.e., where we'd insert either a ↑ or a ← in $T$), we should incur a *gap extension penalty* if the value in $T$ that the new arrow will point to is the same type of arrow. Otherwise, we should incur the *gap open penalty*. If we represent our gap open penalty as $d^0$, and our gap extend penalty as $d^e$, our scoring scheme would now look like the following:

$$
F(i, j) = max \left(\begin{align}
& 0\\
& F(i-1, j-1) + s(c_i, c_j)\\
& \left\{\begin{array}{l l} F(i-1, j) - d^e \quad \text{if $T(i-1, j)$ is ↑}\\ F(i-1, j) - d^o \quad \text{if $T(i-1, j)$ is not ↑} \end{array}  \right\} \\
& \left\{\begin{array}{l l} F(i, j-1) - d^e \quad \text{if $T(i, j-1)$ is ←}\\ F(i, j-1) - d^o \quad \text{if $T(i, j-1)$ is not ←} \end{array}  \right\}
 \end{align}\right)
$$

Notice how we use the gap extend penalty only if the previous max score resulted from a gap in the same sequence because it represents the continuation of an existing gap in that sequence. We know which sequence a gap is being introduced in by the characters in the traceback matrix: ↑ always implies a gap in the sequence on the horizontal axis of $F$ and $T$, and ← always implies a gap in the sequence on the vertical axis of $F$ and $T$.

And here's a quick quiz: is this a Smith-Waterman or Needleman-Wunsch scoring function? How do you know?

Take a look at how the scores differ with these additions:

In [ ]:
## global alignment using affine gap penalty
gap_open_penalty = 8
gap_extend_penalty = 2
aln, score, _ = global_pairwise_align(Protein("HEAGAWGHEE"), Protein("PAWHEAE"), 
                                     gap_open_penalty, gap_extend_penalty, blosum50,penalize_terminal_gaps=True)
print(aln)
print(score)
## local alignment
gap_open_penalty = 8
gap_extend_penalty = 2
aln, score, _ = local_pairwise_align(Protein("HEAGAWGHEE"), Protein("PAWHEAE"), 
                                     gap_open_penalty, gap_extend_penalty, blosum50)
print(aln)
print(score)


## Web services
There are also multiple web services for alignment, for exapmle: https://www.ebi.ac.uk/Tools/emboss/

# Exercises

## Exercise II
Compare the following protein sequences with pairwise aligners and answer the questions! For the sake of simplicity the sequences are provided as a variable. 
For printing the long alignments use the predefined function named as show_tabular_alignment_via_file()

## Exercise II.a 
  * 2a1_vir and 2a2_vir are 2 viral protein sequences.  Use pairwise alignment to dicide if they are related or not!
  * Is local or global alignment the better choice for that?




In [ ]:
seq_vir_2a1 = Protein('PIFLSMFGRAGRNGAKGPRGRRRSPRPPGGSSMTPGDDGNQGPRGPGEQRDQPDQMDPLVHPVTSVRSGPWERLGLRGRGESGVLRETLEMWAGQEGLISRVRDDPRESEVRPVTGVQTVWPE')
seq_vir_2a2 = Protein('MPFYIRSDLGPRRAVRGPRFTVPDPKPPPDPAHPLDGTDNVMMAFPKFKPYGFFAYNPWGPIFLSMFGMAGRNGAKGPRGRRRSPRPLGESSMTPRGRRRPGVQGSRGAEGPAGPDGPAGPFGPSGDVGPMGAAGSQGRRGIRGPQGDPGAVGRTGGVDLKGPGGPAGDMGPAGPRGPAGEAGFVAPESVDMIPPVGFTSATVSAATLNSSKVGPVGDQGIRGPTGPSGAPGSQGPDRDVGGMGPEDTKGDDGPVGPKGPQGATIF')

# here comes your solutions



## Exercise II.b  
  * grip_2b1 contains the 3rd PDZ domain of GRIP1 protein.
  * grip_2b2 contains the middle part of GRIP2 protein.
  * Use pairwise alignment to find the GRIP1 3rd PDZ domain in GRIP2! How similar they are? Which aligner should be used?  



In [ ]:
grip_2b1 = Protein('LVEVAKTPGASLGVALTTSMCCNKQVIVIDKIKSASIADRCGALHVGDHILSIDGTSMEYCTLAEATQFLANTTDQVKLEILPHH')
grip_2b2 = Protein('VRPGGPADREGSLKVGDRLLSVDGIPLHGASHATALATLRQCSHEALFQVEYDVATPDTVANASGPLMVEIVKTPGSALGISLTTTSLRNKSVITIDRIKPASVVDRSGALHPGDHILSIDGTSMEHCSLLEATKLLASISEKVRLEILPVPQSQRPLRPSEAVKVQRSEQLHRWDPCVPSCHSPRPGHCRMPTWATPAGQDQSRSLSSTPFSSPTLNHAFSCNNPSTLPRGSQPMSPRTTMGRRRQRRREHKSSLSLASSTVGPGGQIVHTETTEVVLCGDPLSGFGLQLQGGIFATETLSSPPLVCFIEPDSPAERCGLLQVGDRVLSINGIATEDGTMEEANQLLRDAALAHKVVLEVEFDVAESVIPSSGTFHVKLPKKRSVELGITISSASRKRGEPLIISDIKKGSVAHRTGTLEPGDKLLAIDNIRLDNCPMEDAVQILRQCEDLVKLKIRKDEDNSDELETTGAVSYTVELK')

# here comes your solutions



## Exercise II.c
 * nacc1_2c1 and nacc1_2c2 contains 2 protein sequences with the same 2 domains, but in switched position.
 * Use pairwise alignment to compare the sequences! What do you experience? What other method could be used for these kinds of comparisons?
 



In [ ]:
nacc1_2c1=Protein('MAQTLQMEIPNFGNSILECLNEQRLQGLYCDVSVVVKGHAFKAHRAVLAASSSYFRDLFNNSRSAVVELPAAVQPQSFQQILSFCYTGRLSMNVGDQFLLMYTAGFLQIQEIMEKGTEFFLKVSSPSCDSQGLHAEEAPSSEPQSPVAQTSGWPACSTPLPLVSRVKTEQQESDSVQCMPVAKRLWDSGQKEAGGGGNGSRKMAKFSTPDLAANRPHQPPPPQQAPVVAAAQPAVAAGAGQPAGGVAAAGGVVSGPSTSERTSPGTSSAYTSDSPGSYHNEEDEEEDGGEEGMDEQYRQICNMYTMYSMMNVGQTAEKVEALPEQVAPESRNRIRVRQDLASLPAELINQIGNRCHPKLYDEGDPSEKLELVTGTNVYITRAQLMNCHVSAGTRHKVLLRRLLASFFDRNTLANSCGTGIRSSTNDPRRKPLDSRVLHAVKYYCQNFAPNFKESEMNAIAADMCTNARRVVRKSWMPKVKVLKAEDDAYTTFISETGKIEPDMMGVEHGFETASHEGEAGPSAEALQ')
nacc1_2c2=Protein('MAQTLQMEIPNFGNSILECLNEQRLQGLYGTNVYITRAQLMNCHVSAGTRHKVLLRRLLASFFDRNTLANSCGTGIRSSTNDPRRKPLDSRVLHAVKYYCQNFAPNFKESEMNAIAADMCTNARRVVGDQFLLMYTAGFLQIQEIMEKGTEFFLKVSSPSCDSQGLHAEEAPSSEPQSPVAQTSGWPACSTPLPLVSRVKTEQQESDSVQCMPVAKRLWDSGQKEAGGGGNGSRKMAKFSTPDLAANRPHQPPPPQQAPVVAAAQPAVAAGAGQPAGGVAAAGGVVSGPSTSERTSPGTSSAYTSDSPGSYHNEEDEEEDGGEEGMDEQYRQICNMYTMYSMMNVGQTAEKVEALPEQVAPESRNRIRVRQDLASLPAELINQIGNRCHPKLYDEGDPSEKLELVTRKSWMPKVKCDVSVVVKGHAFKAHRAVLAASSSYFRDLFNNSRSAVVELPAAVQPQSFQQILSFCYTGRLSMNVVLKAEDDAYTTFISETGKIEPDMMGVEHGFETASHEGEAGPSAEALQ')

# here comes your solutions
